# Data Analysis

In [1]:
import pandas as pd

In [2]:
workflows_df = pd.read_pickle("../dumps/workflows_df.pkl")
actions_df = pd.read_pickle("../dumps/actions_df.pkl")
frequent_actions_df = pd.read_pickle("../dumps/frequent_actions_df.pkl")
frequent_actions_noTags_df = pd.read_pickle("../dumps/frequent_actions_noTags_df.pkl")
frequent_docker_commands_subsample_df = pd.read_pickle("../dumps/frequent_docker_commands_subsample_df.pkl")

## Descriptive statistics

### Repo Filtering

- Original [RepoReapers dataset](https://reporeapers.github.io/results/1.html): **1,853,195**
- Repositories classified as containing an _engineered software project_: **446,511** (not available: **38,742**)
- Repositories with _DS-related keywords_ in topics or description: **2516**
- Repositories with at least one workflow: **155**

### Workflows

- Total number of workflows found: **399**
- Valid workflows (valid YAML file): **398**
- Invalid workflows (invalid YAML file): **1**

#### Number of workflows per repository

In [14]:
workflows_df.groupby("repository").count()["filename"].describe()

count    109.000000
mean       3.064220
std        3.567537
min        1.000000
25%        1.000000
50%        2.000000
75%        4.000000
max       26.000000
Name: filename, dtype: float64

#### Most common events triggering workflows

## RQ1 -  "Is GitHub Actions used to automate project deployment?"

- ratio of workflows presenting actions / run commands which relate to Docker
- ratio of workflows that upload a container image to Docker-Hub or to GitHub Packages
- ratio of workflows that upload any type of software package to GitHub Packages.

#### Workflows with keyword `deploy` in name or filename

#### Actions with keyword `deploy` in the slug

- Number of distinct actions used in the dataset
- Number of workflows using at least one of such actions

#### Run commands containing keyword `deploy`

- Number of workflows containing at least one of such commands

#### Workflows with keyword `docker` in name or filename

#### Actions with the keyword `docker` in the slug

- Number of distinct actions used in the dataset
- Number of workflows using at least one of such actions

#### Run commands containing keyword `docker`

- Number of workflows containing at least one of such commands

#### Most common docker commands

#### Workflows with the keyword `package` in name or filename

#### Actions with the keyword `package` in the slug

- Number of distinct actions used in the dataset
- Number of workflows using at least one of such actions

#### Run commands containing keyword `package`

- Number of workflows containing at least one of such commands

## RQ2 - "What are the most frequently used Actions?"

#### Total number of actions used in the dataset

#### Average number of actions per workflow

#### Actions / Run Commands

#### Number of actions available on the GitHub Marketplace
Ratio of actions available in the Marketplace vs custom actions not registered in the marketplace

#### Number of actions by verified creators

#### Distribution of action categories

#### 10 most popular actions

#### 10 most popular actions available on the GitHub Marketplace

## RQ3 - "What are the sets of actions that typically co-occur in workflows"

#### Frequent 2-item sets of Actions

##### With tags

##### Without tags

#### Frequent 3-item sets of Actions

##### With tags

##### Without tags